Disclaimer: I am not a financial advisor. Do not take anything on this code as financial advice, ever.
Do your own research.
Consult a professional investment advisor before making any investment decisions!

This code uses the Markowitz Efficient Frontier to optmize a stock portfolio.
- [Efficient Frontier](https://www.investopedia.com/terms/e/efficientfrontier.asp)
- [Markowitz Efficient Set](https://www.investopedia.com/terms/m/markowitzefficientset.asp)
- [Efficient Frontier - Wikipedia](https://en.wikipedia.org/wiki/Efficient_frontier)

## Dependencies

In [77]:
'''
!pip install investpy
!pip install pandas
!pip install yfinance
!pip install PyPortfolioOpt
'''

'\n!pip install investpy\n!pip install pandas\n!pip install yfinance\n!pip install PyPortfolioOpt\n'

## Imports

In [38]:
import investpy as inv
import pandas as pd
import yfinance as yf
from collections import OrderedDict

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import CovarianceShrinkage, risk_models, expected_returns, objective_functions
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

## Stock List

In [62]:
#lista_tickers = inv.get_stocks_list('brazil')

#source: https://infograficos.valor.globo.com/carteira-valor/
#november-22 

lista_tickers = ['ITUB4','VALE3', 'B3SA3','MULT3','BBDC4','LREN3','BPAC11','ITSA4','ASAI3','ALSO3', 'ANIM3','CYRE3','SBSP3','SOMA3','RENT3','MULT3','IGTI11', 'RDOR3','CCRO3','CIEL3',
'CASH3','AGRO3','HYPE3', 'PRIO3', 'UNIP6', 'ELET3', 'PCAR3', 'RDOR3', 'WEGE3', 'ABEV3', 'LREN3', 'TOTS3','CYRE3', 'GGBR4', 'RADL3', 'COGN3', 'CURY3','MGLU3','KLBN11','MRVE3', 'YDUQ3',
'BBDC4', 'ELET3', 'MULT3', 'STBP3', 'VBBR3', 'CCRO3', 'MDIA3', 'VBBR3', 'ALSO3', 'AURE3', 'BPAC11', 'UGPA3', 'SIMH3', 'CPLE6', 'ODPV3', 'TAEE11', 'HYPE3', 'PSSA3', 'TOTS3', 'AMER3',
'SLCE3', 'GGBR4', 'EMBR3', 'MYPK3', 'TRPL4', 'SLCE3', 'ALSO3', 'EGIE3', 'RANI3', 'RAPT4', 'EQTL3', 'JBSS3' ]


lista_tickers = list(set(lista_tickers))
#lista_tickers = list(OrderedDict.fromkeys(lista_tickers))

len(lista_tickers)

57

## Cleaning Data

In [45]:
df = pd.DataFrame()

for i in lista_tickers:
    ticker = str(i) +'.SA'
    df[i] = yf.download(ticker, start='2019-01-01', end='2022-11-15')['Adj Close']

df2 = df.copy()

df2 = df2.reset_index()

df_bova = df2.melt(id_vars=["Date"],
        var_name="Ticker",
        value_name="Adj_Close")

df_bova = df_bova.dropna()

df_bova.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,Date,Ticker,Adj_Close
0,2019-01-02,ITUB4,32.480240
1,2019-01-03,ITUB4,33.015705
2,2019-01-04,ITUB4,32.462669
3,2019-01-07,ITUB4,32.524113
4,2019-01-08,ITUB4,32.936707


In [47]:
df_bova = pd.pivot_table(df_bova, index=['Date'], columns=['Ticker'], values=['Adj_Close'])

df_bova


Adj_Close                                                        \
Ticker          ABEV3      AGRO3      ALSO3      AMER3     ANIM3      ASAI3   
Date                                                                          
2019-01-02  14.685471  11.282018  25.096451  41.664917  6.001223        NaN   
2019-01-03  14.849149  11.494219  25.594545  42.091255  6.264463        NaN   
2019-01-04  15.049197  11.529586  25.048555  40.695965  6.247803        NaN   
2019-01-07  14.985545  11.529586  25.508337  38.893715  6.294453        NaN   
2019-01-08  14.685471  11.501292  25.335920  40.695965  6.494382        NaN   
...               ...        ...        ...        ...       ...        ...   
2022-11-08  16.740000  29.219999  21.360001  15.170000  6.590000  19.959999   
2022-11-09  16.480000  27.200001  21.090000  13.890000  6.460000  20.400000   
2022-11-10  15.990000  26.400000  19.139999  12.640000  5.880000  19.520000   
2022-11-11  16.090000  27.020000  18.190001  12.210000  5.710000  19.350000   
2022-11-14  15.950000  27.660000  18.309999  12.540000  5.580000  19.600000   

                                                    ...                       \
Ticker      AURE3      B3SA3      BBDC4     BPAC11  ...     STBP3     TAEE11   
Date                                                ...                        
2019-01-02    NaN   7.967909  21.295809   5.700027  ...  3.749658  16.722408   
2019-01-03    NaN   8.314840  21.511545   5.935811  ...  3.855531  16.770758   
2019-01-04    NaN   8.013783  21.342743   6.146415  ...  3.776127  17.433851   
2019-01-07    NaN   7.970774  21.595940   6.272318  ...  3.732013  17.143745   
2019-01-08    NaN   7.993714  21.727819   6.155517  ...  3.723190  17.129930   
...           ...        ...        ...        ...  ...       ...        ...   
2022-11-08  14.20  15.330000  18.580000  28.629999  ...  8.577158  39.381351   
2022-11-09  14.20  15.240000  15.350000  28.150000  ...  8.586872  39.321552   
2022-11-10  13.67  13.310000  14.900000  26.129999  ...  7.693216  38.264999   
2022-11-11  13.78  12.580000  15.150000  25.670000  ...  7.530000  39.391323   
2022-11-14  13.83  12.470000  15.160000  26.250000  ...  7.770000  40.128910   

                                                                               \
Ticker          TOTS3      TRPL4      UGPA3       UNIP6      VALE3      VBBR3   
Date                                                                            
2019-01-02   9.261152  13.173595  24.992414   20.529421  37.180233  19.462721   
2019-01-03   9.474497  13.361790  24.527887   20.284695  35.659252  19.911863   
2019-01-04   9.736332  13.851096  24.362314   20.279257  37.980740  19.986713   
2019-01-07   9.526217  13.738178  23.617229   20.230314  37.776974  20.099001   
2019-01-08   9.138315  13.698658  23.226294   20.801329  38.140846  19.425297   
...               ...        ...        ...         ...        ...        ...   
2022-11-08  32.099998  23.990000  13.530000  107.895309  74.050003  18.530001   
2022-11-09  33.389999  23.959999  13.560000  108.601761  73.150002  18.410000   
2022-11-10  31.570000  23.240000  13.240000  103.036110  74.550003  17.180000   
2022-11-11  31.160000  23.650000  13.960000  104.105324  82.300003  17.020000   
2022-11-14  32.660000  23.700001  13.720000  108.830872  83.309998  17.059999   

                                  
Ticker          WEGE3      YDUQ3  
Date                              
2019-01-02   8.541142  23.391680  
2019-01-03   8.735907  23.372854  
2019-01-04   8.835665  23.730553  
2019-01-07   8.697903  23.259893  
2019-01-08   8.626649  23.476398  
...               ...        ...  
2022-11-08  40.799999  13.980000  
2022-11-09  40.599998  13.720000  
2022-11-10  39.230000  11.810000  
2022-11-11  39.990002  12.800000  
2022-11-14  40.660000  12.480000  

[961 rows x 57 columns]

In [48]:
d2 = pd.DataFrame(df_bova.to_records())
d2.columns = d2.columns.str.replace("'Adj_Close',","")
d2.columns = d2.columns.str.replace("(","")
d2.columns = d2.columns.str.replace(")","")
d2.columns = d2.columns.str.replace("'","")
d2

In [52]:
#ajuste do indice
df2 = d2.set_index(pd.DatetimeIndex(d2['Date'].values))
df2.drop(columns=['Date'], axis=1, inplace=True)

In [54]:
df2

,ABEV3,AGRO3,ALSO3,AMER3,ANIM3,ASAI3,AURE3,B3SA3,BBDC4,BPAC11,...,STBP3,TAEE11,TOTS3,TRPL4,UGPA3,UNIP6,VALE3,VBBR3,WEGE3,YDUQ3
2019-01-02,14.685471,11.282018,25.096451,41.664917,6.001223,NaN,NaN,7.967909,21.295809,5.700027,...,3.749658,16.722408,9.261152,13.173595,24.992414,20.529421,37.180233,19.462721,8.541142,23.391680
2019-01-03,14.849149,11.494219,25.594545,42.091255,6.264463,NaN,NaN,8.314840,21.511545,5.935811,...,3.855531,16.770758,9.474497,13.361790,24.527887,20.284695,35.659252,19.911863,8.735907,23.372854
2019-01-04,15.049197,11.529586,25.048555,40.695965,6.247803,NaN,NaN,8.013783,21.342743,6.146415,...,3.776127,17.433851,9.736332,13.851096,24.362314,20.279257,37.980740,19.986713,8.835665,23.730553
2019-01-07,14.985545,11.529586,25.508337,38.893715,6.294453,NaN,NaN,7.970774,21.595940,6.272318,...,3.732013,17.143745,9.526217,13.738178,23.617229,20.230314,37.776974,20.099001,8.697903,23.259893
2019-01-08,14.685471,11.501292,25.335920,40.695965,6.494382,NaN,NaN,7.993714,21.727819,6.155517,...,3.723190,17.129930,9.138315,13.698658,23.226294,20.801329,38.140846,19.425297,8.626649,23.476398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-08,16.740000,29.219999,21.360001,15.170000,6.590000,19.959999,14.20,15.330000,18.580000,28.629999,...,8.577158,39.381351,32.099998,23.990000,13.530000,107.895309,74.050003,18.530001,40.799999,13.980000
2022-11-09,16.480000,27.200001,21.090000,13.890000,6.460000,20.400000,14.20,15.240000,15.350000,28.150000,...,8.586872,39.321552,33.389999,23.959999,13.560000,108.601761,73.150002,18.410000,40.599998,13.720000
2022-11-10,15.990000,26.400000,19.139999,12.640000,5.880000,19.520000,13.67,13.310000,14.900000,26.129999,...,7.693216,38.264999,31.570000,23.240000,13.240000,103.036110,74.550003,17.180000,39.230000,11.810000
2022-11-11,16.090000,27.020000,18.190001,12.210000,5.710000,19.350000,13.78,12.580000,15.150000,25.670000,...,7.530000,39.391323,31.160000,23.650000,13.960000,104.105324,82.300003,17.020000,39.990002,12.800000


## Efficient Frontier

In [73]:

mean = expected_returns.mean_historical_return(df2)

s = CovarianceShrinkage(df2).ledoit_wolf()

In [74]:
#calcular Sharp ratio

ef = EfficientFrontier(mean, s)

#para não alocar recursos nas ações que estão zeradas e manter apenas as relevantes
ef.add_objective(objective_functions.L2_reg, gamma = 0.1)

w = ef.max_sharpe()

cleaned_weights  = ef.clean_weights()

print(cleaned_weights )

OrderedDict([(' ABEV3', 0.0), (' AGRO3', 0.036), (' ALSO3', 0.0), (' AMER3', 0.0), (' ANIM3', 0.0), (' ASAI3', 0.16309), (' AURE3', 0.0), (' B3SA3', 0.0), (' BBDC4', 0.0), (' BPAC11', 0.0), (' CASH3', 0.0), (' CCRO3', 0.0), (' CIEL3', 0.0), (' COGN3', 0.0), (' CPLE6', 0.06371), (' CURY3', 0.03492), (' CYRE3', 0.0), (' EGIE3', 0.0), (' ELET3', 0.0), (' EMBR3', 0.0), (' EQTL3', 0.0), (' GGBR4', 0.0), (' HYPE3', 0.0), (' IGTI11', 0.0), (' ITSA4', 0.0), (' ITUB4', 0.0), (' JBSS3', 0.0), (' KLBN11', 0.0), (' LREN3', 0.0), (' MDIA3', 0.0), (' MGLU3', 0.0), (' MRVE3', 0.0), (' MULT3', 0.0), (' MYPK3', 0.0), (' ODPV3', 0.0), (' PCAR3', 0.0), (' PRIO3', 0.25404), (' PSSA3', 0.0), (' RADL3', 0.00103), (' RANI3', 0.01659), (' RAPT4', 0.0), (' RDOR3', 0.0), (' RENT3', 0.0), (' SBSP3', 0.0), (' SIMH3', 0.0), (' SLCE3', 0.03355), (' SOMA3', 0.0), (' STBP3', 0.0), (' TAEE11', 0.11996), (' TOTS3', 0.00557), (' TRPL4', 0.0), (' UGPA3', 0.0), (' UNIP6', 0.11746), (' VALE3', 0.0), (' VBBR3', 0.0), (' WEG

c:\Python310\lib\site-packages\pypfopt\efficient_frontier\efficient_frontier.py:257: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(


In [75]:
ef.portfolio_performance(verbose=True)

Expected annual return: 56.1%
Annual volatility: 28.1%
Sharpe Ratio: 1.92


(0.5614886169475294, 0.2813634610370944, 1.924516477554065)

In [76]:
latest_prices = get_latest_prices(df2)
da = DiscreteAllocation(w, latest_prices, total_portfolio_value=20000)
allocation, leftover = da.lp_portfolio()
print('Qtd ações alocadas: ',allocation)
print('Sobrou: R$',leftover)

Qtd ações alocadas:  {' AGRO3': 26, ' ASAI3': 166, ' CPLE6': 172, ' CURY3': 60, ' PRIO3': 132, ' RADL3': 1, ' RANI3': 38, ' SLCE3': 15, ' TAEE11': 59, ' TOTS3': 3, ' UNIP6': 22, ' WEGE3': 76}
Sobrou: R$ 5.279059432454233
